# Annotate new data to improve Misogyny Detection models using Rubrix and biome.text

Hey there! In this guide, we will show you how we used Rubrix to annotate new data and use this new data to improve existing Deep Learning models. Our use case was Automatic Misogyny Detection (AMI): NLP models able to detect the underlying misogyny on a given text. Ground-breaking work is being made every year on this subject, with shared tasks and new models that push the performance of these models closer and closer to be implemented in apps, social networks, and other digital environments. 

Alongside Rubrix, we used our amazing NLP libary [biome.text](https://github.com/recognai/biome-text) to train models with a simple workflow. Rubrix is compatible with almost any library or service, so we were able to work back and forth with both of them. 

## Our objective

One of our team members, Ignacio, was preparing his Bachelor's thesis on the subject of Automatic Misogyny Detection. In this setting, we wanted to use the potential of Rubrix on our favor, going beyond the classic linear workflows that are so common in Deep Learning: gathering some data, preprocessing it, training a model and start making inference. Rubrix breaks that scheme, and allowed us to use a man-in-the-middle approach with retraining and fine-tuning and to add new data in follow-up trainings. 

We want to talk you more about it in this guide. It is not intended to be a tutorial, even though there will be snippets of code to reproduce our process and get to similar result (we will simplify some aspects of it to keep it light-weighted), but more of a story. We will focus on the process and on what we learnt along the way. 

## Background

The Bachelor's thesis of Ignacio was focused on the Spanish language, and datasets of misogyny in Spanish text is nothing but scarce. This was the first hardship: finding datasets to start training some models. Luckily, there is a very important community of shared task focused on the matter, covering a lot of non-English languages. We started working with data from [IberEval 2018](https://sites.google.com/view/ibereval-2…), a shared-task that offered a compilation of tweets, analyzed by experts and classified into 5 different misogyny categories. We started training our first model with around three thousand instances of annotated data. 

## Appendix

Here are some procedures we've made for this guide that were kept on the background. If you want to reproduce all our steps, including the training of models and some extra parts, we will give provide with cells to do so! Feel free to change anything and try new stuff, and tell us if you have some doubts our find something cool at our [Github forum](https://github.com/recognai/rubrix/discussions)

### Training our first model

To reproduce a simplified version of the first trained model, before annotation, you can execute the following cells. We've already searched for good-enough configurations, so you can skip that step.

Let's start by loading the datasets

In [ ]:
# Loading the datasets
training_ds = Dataset.from_csv('annotation_data/training_full_df.csv', index=False)
test_ds = Dataset.from_csv('annotation_data/test_df.csv', index=False)

Creating NLP pipelines with biome.text is quick and convenient! We performed an HPO process on the background, to find suitable hyperparameters for this domain, so let's use them to create our first AMI model. Note that we're making a pipeline with BETO, a Spanish Transformer model, at the head. To learn more about what a Transformer is, please visit the [Transformer guide of biome.text](https://recognai.github.io/biome-text/v3.0.0/documentation/tutorials/4-Using_Transformers_in_biome_text.html).

In [ ]:
pipeline_dict = {
    "name": "AMI_first_model",
    "features": {
        "transformers": {
            "model_name": "dccuchile/bert-base-spanish-wwm-cased", # BETO model
            "trainable": True,
            "max_length": 280,  # As we are working with data from Twitter, this is our max length
        }
    },
    "head": {
        "type": "TextClassification",
        
        # These are the possible misogyny categories.
        "labels": [
            'sexual_harassment',
             'dominance',
             'discredit',
             'stereotype',
             'derailing',
             'non-sexist'
        ],
        "pooler": {
            "type": "lstm",
            "num_layers": 1,
            "hidden_size": 256,
            "bidirectional": True,
        },
    },
}

pl = Pipeline.from_config(pipeline_dict)

In [ ]:
trainer_config = TrainerConfiguration(
    optimizer={
        "type": "adamw",
        "lr": 0.000023636840436059507,
        "weight_decay": 0.01438297700463013,
    },
    batch_size=8,
    max_epochs=10,
)

In [ ]:
trainer = Trainer(
    pipeline=pl,
    train_dataset=training_ds,
    valid_dataset=test_ds,
    trainer_config=trainer_config
)

In [ ]:
trainer.fit()

After `trainer.fit()` stops, the results of the training and the obtained model will be in the output folder.

We can make some predictions, and take a look at the performance of the model.

In [ ]:
pl.predict("Las mujeres no deberían tener derecho a voto")